In [19]:
import pinocchio as pin
import os
from pinocchio.visualize import MeshcatVisualizer
import numpy as np
from numpy.linalg import solve,norm
# from robot_descriptions.loaders.pinocchio import load_robot_description
# import gepetto.corbaserver

In [20]:
urdf_filename = "/home/adi/hum_rob_ws/src/six_dof/urdf/6dof_from_hip.urdf"
mesh_dir = "/home/adi/hum_rob_ws/src/six_dof/meshes"

ros_package_path = "/home/adi/hum_rob_ws/src"  # Replace with your path
os.environ["ROS_PACKAGE_PATH"] = ros_package_path

In [21]:
# building model from URDF, model is an object that includes kinematic and ineratia params
model = pin.buildModelFromUrdf(urdf_filename)

visual_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.VISUAL, package_dirs=mesh_dir
)  # very important

collision_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.COLLISION, package_dirs=mesh_dir
)

geom_data = pin.GeometryData(collision_model)


# data is an object that holds values that are a result of computation
data = model.createData()

# print('standard model: dim=' + str(len(model.joints)))
# for jn in model.joints:
#     print(jn)
# print('-' * 30)


# # Create a list of joints to lock
# jointsToLock = ['right_foot_shin', 'right_shin_thigh', 'right_thigh_body']
 
# # Get the ID of all existing joints
# jointsToLockIDs = []
# for jn in jointsToLock:
#     if model.existJointName(jn):
#         jointsToLockIDs.append(model.getJointId(jn))
#     else:
#         print('Warning: joint ' + str(jn) + ' does not belong to the model!')

# print(jointsToLockIDs)
 
# geom_models = [visual_model, collision_model]
# model_reduced, geometric_models_reduced = pin.buildReducedModel(
#     model,
#     list_of_geom_models=geom_models,
#     list_of_joints_to_lock=jointsToLockIDs,
#     reference_configuration=pin.neutral(model))
# # geometric_models_reduced is a list, ordered as the passed variable "geom_models" so:
# visual_model_reduced, collision_model_reduced = geometric_models_reduced[
#     0], geometric_models_reduced[1]

In [22]:
visualizer = MeshcatVisualizer(model, collision_model, visual_model)


visualizer.initViewer()
visualizer.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


In [23]:
q = pin.neutral(model)
print(q)
eps = 1e-3
DT = 1e-1
damp = 1e-1
IT_MAX = 20000
JOINT_ID = 8


R_des = np.array([[1, 0, 0], [0, 0.990216, -0.139543], [0, 0.139543, 0.990216]])
R_des=np.eye(3)

oMdes = pin.SE3(
    R_des, np.array([0.0235, 0.18, -0.15])
)  # think of oMdes as oTdes, the transfromation from desired to origin expressed in origin
i = 0

[0. 0. 0. 0. 0. 0. 0. 0.]


In [24]:
visualizer.display(q)
visualizer.viewer.jupyter_cell()

In [25]:
while True:
    pin.forwardKinematics(model, data, q)
    
    iMd = data.oMi[JOINT_ID].actInv(oMdes) #active inverse, chnages basis and moves the vector/point to the appropriate position in the new basis
    err = pin.log(iMd).vector  # in joint frame  The logarithm map (log()) is a mathematical operation that maps elements from the group SE(3) to its associated Lie algebra, se(3). This operation effectively converts the complex rotational and translational transformation into a simpler vector representation that captures the "difference" or "error" between two poses in SE(3).
    norm_err=norm(err)
    if norm_err < eps:
        success = True
        break
    if i >= IT_MAX:
        success = False
        break
    J = pin.computeJointJacobian(
        model, data, q, JOINT_ID
    )  # jacobian computed in e-e frame
    J = -np.dot(pin.Jlog6(iMd.inverse()), J)
    q_dot = -J.T.dot(solve(J.dot(J.T) + damp * np.eye(6), err))
    q = pin.integrate(model, q, q_dot * DT)
    
    #Collision check
    pin.updateGeometryPlacements(
        model, data, collision_model, geom_data, q
    )  # updates geom_data by reference

    # an active pair is a pair of bodies in a joint considered for collision
    collision_detected = pin.computeCollisions(
        model, data, collision_model, geom_data, q, True
    )  # this returns data,geom_data. polymorphic function

    if not collision_detected:
        for idx in range(model.nq):
            # Assuming model.lowerPositionLimit and model.upperPositionLimit store the limits
            q[idx] = max(
                model.lowerPositionLimit[idx],
                min(q[idx], model.upperPositionLimit[idx]),
            )
    
    
        visualizer.display(q)
    print(norm_err)
    #print(i)
    i+=1
    #time.sleep(0.05)

if success:
    print("Convergence achieved!")
else:
    print(
        "\nWarning: the iterative algorithm has not reached convergence to the desired precision"
    )

print("\nresult: %s" % q.flatten().tolist())
print("\nfinal error: %s" % err.T)



0.14035733972155018
0.1270148684496686
0.11498057872146278
0.10412875995225387
0.09434593466183623
0.08552967258030797
0.0775875175601766
0.07043601597841416
0.06399983632697623
0.05821097062890489
0.05300800920019323
0.048335481129202655
0.04414325369229413
0.04038598479340641
0.03702262342929874
0.034015954152337415
0.031332182519210126
0.028940559535255306
0.02681304404905124
0.024924002800008914
0.023249948227895355
0.021769314080813854
0.020462268227233266
0.01931056091533485
0.018297405194341636
0.01740738460978206
0.016626381965258806
0.01594152223119366
0.015341122766878958
0.014814644904050595
0.014352642430049849
0.013946704299388568
0.01358939066467253
0.013274162786175032
0.012995308402223075
0.012747864687181728
0.012527541049105218
0.012330643834667247
0.012154004636774665
0.01199491344799578
0.011851057450866296
0.011720465834767931
0.01160146070336394
0.011492613892761381
0.011392709353153216
0.011300710644077911
0.011215733041958515
0.011137019744984763
0.0110639216731

In [26]:
pin.forwardKinematics(model,data,q)
 
# Print out the placement of each joint of the kinematic tree
for name, oMi in zip(model.names, data.oMi):
    print(("{:<24} : {: .3f} {: .3f} {: .3f}"
          .format( name, *oMi.translation.T.flat )))



universe                 :  0.000  0.000  0.000
body_left_thigh_lateral  :  0.100  0.000  0.000
body_left_thigh          :  0.100  0.000  0.000
left_thigh_shin          :  0.095 -0.000 -0.160
left_shin_foot           :  0.122 -0.000 -0.290
right_body_thigh_lateral :  0.000  0.000  0.000
right_body_thigh         :  0.041  0.000  0.000
right_thigh_shin         :  0.044 -0.036 -0.156
right_shin_foot          :  0.024 -0.000 -0.281


In [27]:
result_deg=[np.rad2deg(i) for i in q]
print(result_deg)

[0.0, 0.0, 0.0, 0.0, 0.0012483245013288932, -12.82653684026712, 28.781336591000724, -7.932643510685591]
